In [1]:
%config Application.log_level="INFO"
from pandas_datareader import wb
df2 = wb.download(indicator='NY.GDP.PCAP.KD', country=['all'], start=2005, end=2008)
df2.columns = ['GDP per capita (constant 2005 US$)']
df2

GDP per capita (constant 2005 US$)
country                                     year                                    
Arab World                                  2008                         5898.516152
                                            2007                         5711.663247
                                            2006                         5594.898519
                                            2005                         5378.379169
Caribbean small states                      2008                         9007.290662
                                            2007                         8938.276087
                                            2006                         8715.446097
                                            2005                         8210.932496
Central Europe and the Baltics              2008                        12720.215011
                                            2007                        12212.955838
                                            2006                        11435.871982
                                            2005                        10703.620850
Early-demographic dividend                  2008                         2801.923940
                                            2007                         2744.578123
                                            2006                         2620.119100
                                            2005                         2500.834773
East Asia & Pacific                         2008                         7172.877579
                                            2007                         6979.228960
                                            2006                         6599.877511
                                            2005                         6299.476097
East Asia & Pacific (excluding high income) 2008                         3440.505855
                                            2007                         3197.589510
                                            2006                         2868.968984
                                            2005                         2610.445627
East Asia & Pacific (IDA & IBRD countries)  2008                         3477.541272
                                            2007                         3232.070890
                                            2006                         2899.948621
                                            2005                         2638.645640
Euro area                                   2008                        38800.519930
                                            2007                        38822.529747
...                                                                              ...
Vanuatu                                     2006                         2777.665956
                                            2005                         2625.269068
Venezuela, RB                               2008                        14652.178425
                                            2007                        14143.657950
                                            2006                        13221.420660
                                            2005                        12237.882953
Vietnam                                     2008                         1214.252458
                                            2007                         1161.471573
                                            2006                         1095.982452
                                            2005                         1035.922170
Virgin Islands (U.S.)                       2008                        43010.440884
                                            2007                        42477.501084
                                            2006                        40724.818541
                                            2005                        39240.488334
West Bank and Gaza                          2008       

In [2]:
import qgrid
q = qgrid.QgridWidget(df=df2)
q

A Jupyter Widget

In [3]:
import pandas as pd
import numpy as np

df3 = pd.DataFrame(np.random.randn(1000000, 4), columns=list('ABCD'))

In [4]:
df3['B'] = df3['B'].apply(lambda x: str(x))

In [5]:
%config Application.log_level="INFO"
import qgrid
q = qgrid.QgridWidget(df=df3)
q

A Jupyter Widget

### 2. Render the DataFrame again, this time using qgrid

In [ ]:
vbox_exported = qgrid.show_grid(df2, remote_js=True)
vbox_exported

In [ ]:
vbox.children[1]

## Example 3 - Rendering a grid by creating a QGridWidget

In [ ]:
df3.dtypes

### 1. Create a new sample DataFrame and render it without using qgrid

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.html.table_schema', False)

td = np.cumsum(np.random.randint(1, 15*60, 1000))
start = pd.Timestamp('2017-04-17')
df = pd.DataFrame(
    [(start + pd.Timedelta(seconds=d)) for d in td],
    columns=['time'])

freq = '15Min'
start = df['time'].min().floor(freq)
end = df['time'].max().ceil(freq)
bins = pd.date_range(start, end, freq=freq)

df['time_bin'] = pd.cut(df['time'], bins)
df

In [ ]:
df[df.index.isin([2,3])]

In [ ]:
[1].first()

In [3]:
import pandas as pd
import numpy as np

df3 = pd.DataFrame({
    'A' : 1.,
    'B' : pd.Timestamp('20130102'),
    'C' : pd.Series(np.random.randn(8),index=list(range(8)),dtype='float64'),
    'D' : np.array([3] * 8,dtype='int32'),
    'E' : pd.Categorical(["washington", "adams", "adams", "lincoln","jefferson", "hamilton", "roosevelt", "kennedy"]),
    'F' : 'foo' })
df3

,A,B,C,D,E,F
0,1.0,2013-01-02,-1.265327,3,washington,foo
1,1.0,2013-01-02,0.776191,3,adams,foo
2,1.0,2013-01-02,-0.123946,3,adams,foo
3,1.0,2013-01-02,-0.795014,3,lincoln,foo
4,1.0,2013-01-02,1.437033,3,jefferson,foo
5,1.0,2013-01-02,0.129964,3,hamilton,foo
6,1.0,2013-01-02,-0.294109,3,roosevelt,foo
7,1.0,2013-01-02,-0.640640,3,kennedy,foo


In [4]:
import qgrid
q = qgrid.QgridWidget(df=df3)
q

A Jupyter Widget

In [ ]:
from operator import le, ge
condition1 = df3.index.isin([1,2,3])
condition2 = df3['E'].isin(['adams'])
condition3 = df3.index.isin([2])
conditions = [condition1, condition2, condition3]
# def match(df):
#     combined_condition = (conditions[0])
#     for c in conditions[1:]:
#         combined_condition = combined_condition) & (c(df))
#     return combined_condition

# def b(x, col, op, n): 
#     return op(x[col],n)

# def f(x, *b):
#     return x[(np.logical_and(*b))]

# b1 = b(df, 'col1', ge, 1)
# b2 = b(df, 'col1', le, 1)
combined_condition = conditions[0]
for c in conditions[1:]:
    combined_condition = combined_condition & c
print(combined_condition)
df3[combined_condition]
# df3[eval('conditions[0] & conditions[1]')]

In [ ]:
isin

In [ ]:
df3.query('(E == "adams" )')

In [ ]:
df3.copy()

In [ ]:
x = 123
x > -0.23

### 2. Render the DataFrame again, this time using the QGridWidget class directly
The ``show_grid`` function is just a convenience function which internally constructs an instance of QGridWidget and renders it with jupyter notebook's ``display`` function.  The following code shows how to construct a QGridWidget directly, so that you can retrieve or update the DataFrame it uses internally.

In [ ]:
import qgrid
pd.options.mode.chained_assignment = None
grid = qgrid.QgridWidget(df=df3)
grid

In [ ]:
df3.loc[:, 'D']

### 3. Get the DataFrame back from the QGridWidget and render it without qgrid
If you make edits to the data using the grid above, they will be reflected in the DataFrame that is returned by ``grid.df``.

In [ ]:
grid.df